<a href="https://colab.research.google.com/github/cuauhtemocbe/Python-Basics/blob/main/1_Generalidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generalidades texto (update)




In [ ]:
# ! jupyter nbconvert --to html 1_Generalidades.ipynb

In [ ]:
import missingno as msno
import pandas as pd

%matplotlib inline

pd.set_option('display.max_columns', None)

**Objetivo**

Desarrollar un modelo predictivo que nos permita predecir, dados atributos climatologicos, si va a llover o no.

**Introducción**

Nuestro cliente, el gobierno de Australia, nos ha solicitado un modelo de clasificación, el cual nos permita identificar si dadas ciertas condiciones climatologicas, va a llover o no. Para ello nos han proporcionado un dataset y nos han solicitado en un tiempo menor a 48 horas.



**Metodología** <br>
Debido a la premura del tiempo y al desconocimiento en modelos climatologicos, el primer paso es investigar un poco sobre las condiciones climatologicas y la distribución de la lluvia en Australia.  <br>

Posteriormente se desarrollará un modelo base, que nos permita tener un mejor entendimiento de los datos, además de darnos la posibilidad de tener un entregable en la fecha solicitada. <br>

Después se profundizará en los atributos, para mejorar su procesamiento, seleccionar los mejores, o desarrollar otras métricas.

También se buscará generar otros tipos de modelos, que sean locales, para poder compararlo con el modelo base.

Entonces, la estragia será el desarrollo de un modelo 'Nacional' y uno local para 'Sydney' e identificar áreas de mejora.

**Área de estudio**

Los datos corresponden a datos atmosfericos y con un enfoque en la predicción de la variable objetivo: **RainTomorrow**, con lo que buscamos predecir, dadas ciertas condiciones si lloverá.

En el siguiente mapa se observa la distribución de la lluvia en Australia. Existe una mayor cantidad de precipitación en las costas, siendo menor en las zona central del país. 

Por esto podría ser conveniente realizar un estudio local o regional, para la obtención de un modelo.

In [ ]:
from IPython.display import Image
Image(r"../img/distribucion_lluvia_AU.jpg")

Otro factor a considerar, es la temporalidad de la lluvia, ya que de acuerdo al gobierno de Australia ocurre de noviembre a mayo.

## Descripción de los atributos

**0. Date:** Fecha del registro <br>
**1. Location:** Nombre del lugar de la estación climatica <br>
**2. MinTemp:** Temperatura mínima (°C) <br>
**3. MaxTemp:** Temperatura máxima (°C) <br>
**4. Rainfall:** Cantidad de lluvia registrada (mm) <br>
**5. Evaporation:** Evaporación en 24 h (mm) <br>
**6. Sunshine:** Horas luz en el día (h) <br>
**7. WindGustDir:** La dirección del viento predominante en 24h<br>
**8. WindGustSpeed:** Velocidad del viento predominante en 24h (km/h) <br>
**9. WindDir9am:** Dirección del viento a las 9 am <br>
**10. WindDir3pm:** Dirección del viento a las 3 pm <br>
**11. WindSpeed9am:** Velocidad promedio del viento en 10 min a las 9 am (km/hr) <br>
**12. WindSpeed3pm:** Velocidad promedio del viento en 10 min a las 3 pm (km/hr) <br>
**13. Humidity9am:** Humedad a las 9 am (%) <br>
**14. Humidity3pm:** Humedad a las 3 pm (%) <br>
**15. Pressure9am:** Presión atmosférica a las 9 am (hpa) <br>
**16. Pressure3pm:** Presión atmosférica a las 3 pm (hpa) <br>
**17. Cloud9am:** Nubosidad a las 9 am (oktas). Más cercano a 0 es cielo despejado, más cercano a 8 es totalmente nublado <br>
**18. Cloud3pm:** Nubosidad a las 3 pm (oktas). Más cercano a 0 es cielo despejado, más cercano a 8 es totalmente nublado <br>
**19. Temp9am:** Temperatura a las 9 am (°C) <br>
**20. Temp3pm:** Temperatura a las 3 pm (°C) <br>
**21. RainToday:** 1 si la precipitación excede 1 mm en 24 horas, de otro modo 0 <br>
**22. RainTomorrow:** Cantidad de lluvia al siguiente día (mm). Medida de "riesgo"<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_weather(path: str) -> pd.DataFrame:
    """Reads the csv file with the weather data from Australia,
    and the column "Date" is processed"""
    df = pd.read = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    
    return df

In [ ]:
# Leyendo el dataset
df = read_weather(r'/content/drive/MyDrive/Colab Notebooks/Precipitation-Australia/data/weatherAUS.csv')

In [ ]:
df.shape

(145460, 25)

El conjunto de datos contiene 145, 460 registros y 23 atributos (generé 2, correspondientes al año (year) y al mes (month).

In [ ]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,2008,12
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,2008,12
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,2008,12
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,2008,12


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  object        
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    135134 non-null  object        
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     134894 non-null  object        
 10  WindDir3pm     141232 non-null  object        
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

Observamos que contamos con dos tipos de datos: datetime, object y float.

In [ ]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,year,month
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000,145460.000000,145460.000000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339,2012.769751,6.399615
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665,2.537684,3.427262
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000,2007.000000,1.000000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000,2011.000000,3.000000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000,2013.000000,6.000000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000,2015.000000,9.000000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000,2017.000000,12.000000


También notemos que tenemos datos faltantes los cuales se analizarán a profundidad en el siguiente notebook.

In [ ]:
round(df.groupby('RainTomorrow')['RainTomorrow'].count() / df['RainTomorrow'].count() * 100, 2)

RainTomorrow
No     77.58
Yes    22.42
Name: RainTomorrow, dtype: float64

En los datos revisados, podemos observar que existen un 22.42% de registros con la etiqueta "Yes", esto que se llovió, y el 77.58% de los datos con la etiqueta "No", esto es que no llovió.

Podríamos revisar la distribución de las etiquetas en la temporada de lluvias en Australia, que es de noviembre a mayo, lo que nos podría ayudar a aumentar el número de etiquetas de "Yes"

**Resultados obtenidos**

- Para el modelo **'Nacional'** obtuve un 61% con la métrica F1-score usando el algoritmo XGBoost
- Para el modelo **'Sydney** obtuve un 67% con la métrica F1-score usando el algoritmo Regresión Logística

**Conclusiones generales**

- El modelo base 'Nacional' cuenta con un desempeño aceptable, con un 61% en F1. 
- Aunque probé distintos algoritmos de clasificación, además de una ingeniería para la selección de atributos, no fue posible superar el 61% en F1 en el modelo 'Nacional', lo que me lleva a pensar que es necesario mejorar el procesamiento y análisis de los datos.
- Los modelos locales son más prometedores que un modelo 'Nacional'. 

**Recomendaciones:**

- Aún quedan muchos análisis que realizar y ajustes para mejorar el modelo, como por ejemplo: un mejor manejo en las variables categóricas, profundizar en la distribución de la información, análisis y procesamiento de los datos a nivel local e incluso un análisis enfocado en la temporada de lluvias.
- Para un enfoque regional, podría llevarse a cabo una clusterización que agrupe las ciudades con condiciones climatológicas similares.
- Mejorar el conocimiento del fénomeno de precipitación en Australia
- También podría mejorarse el desempeño del modelo a través de técnicas de ensamblado de modelos.